# Further Analyses- Producing 95% CIs and Examining Most Important Features

In [1]:
from IPython.core.interactiveshell import InteractiveShell
from matplotlib import pyplot 
from numpy import mean
from numpy import std
from sklearn.calibration import calibration_curve
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score as ppv
from sklearn.metrics import recall_score as recall
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
import joblib
import numpy as np
import pandas as pd
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Prepare 100 bootstrapped test sets, and obtain estimates of performance metrics for each resampled version of test data
n_iterations=100
XGB95aucs=list()
XGB95f1s=list()
XGB95sens=list()
XGB95specs=list()
XGB95ppvs=list()
for i in range(n_iterations):
    test=resample(values, n_samples=len(test56), stratify=values[:, -1], random_state=i)
    XGB95probs=cXGB.predict_proba(test[:, :-1])
    XGB95probs=XGB95probs[:, 1]
    XGB95auc=roc_auc_score(test[:, -1], XGB95probs)
    XGB95aucs.append(XGB95auc)
    XGB95pred=np.where(XGB95probs>= 0.06723701432347298, 1, 0)
    XGB95f1=f1_score(test[:, -1], XGB95pred)
    XGB95f1s.append(XGB95f1)
    XGB95sen=recall(test[:, -1], XGB95pred)
    XGB95sens.append(XGB95sen)
    XGB95ppv=ppv(test[:, -1], XGB95pred)
    XGB95ppvs.append(XGB95ppv)
    tn, fp, fn, tp=confusion(test[:, -1], XGB95pred).ravel()
    XGB95spec=tn/(tn+fp)
    XGB95specs.append(XGB95spec)